In [235]:
import pandas as pd
df = pd.read_csv('recipes_data.csv')

Index(['title', 'ingredients', 'directions', 'link', 'source', 'NER', 'site'], dtype='object')

In [236]:
import ast
# clean up NER
df['NER'] = df['NER'].apply(lambda x: set(ast.literal_eval(x)))
df = df[df['NER'] != set()]

In [253]:
# df[df['NER'].apply(lambda x: x == set())]
df

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"{milk, brown sugar, nuts, vanilla, butter}",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"{beef, cream of mushroom soup, sour cream, chi...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"{salt, butter, garlic powder, pepper, frozen c...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"{shredded cheese, chicken gravy, cream of mush...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"{peanut butter, butter, chocolate chips, powde...",www.cookbooks.com
...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"{hazelnuts, tortillas, butter, marshmallows}",www.foodnetwork.com
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"{paprika, relish, salt, eggs}",cookpad.com
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"{salt, white sesame seeds, sesame oil, soy sau...",cookpad.com
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"{bay leaves, horseradish, caraway seeds, coria...",cooking.nytimes.com


In [35]:
df['directions'] = df['directions'].apply(lambda x: ast.literal_eval(x))

In [237]:
import re
def remove_non_alphabet(s):
    return {re.sub(r'[^a-zA-Z ]', '', word) for word in s}

df['NER'] = df['NER'].apply(remove_non_alphabet)

In [238]:
import requests
import json
import re
api_url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
app_id = '4a922d3b'
app_key = '581d2bca47024d1740305fde31226677'
query = '1 cup rice'
headers = {
    'Content-Type' : 'application/json',
    "x-app-id": app_id,
    "x-app-key": app_key
}
params = {
    "query": query
}
response = requests.post(api_url, json=params, headers=headers)
if response.status_code == 200:
    data = response.text
    food_names = re.findall(r'"food_name":"(.*?)"', data)
    print(food_names)
else:
    print(f'Request failed: Error {response.status_code}')

Request failed: Error 401


In [239]:
# drops = {}
count = 0
def check_ingredients(s, ings):
    global count
    food_items = s.intersection(ings)
    if len(food_items) < len(s):
        count += 1
    return food_items

In [240]:
import inflect

p = inflect.engine()
with open('train.json', 'r') as file:
    data = json.load(file)

ings = set()
for recipe in data:
    s = recipe["ingredients"]
    ings.update(s)

opposite = set()
for x in ings:
    alt = None
    if p.singular_noun(x):
        alt = p.singular_noun(x)
    else:
        alt = p.plural(x)
    if alt:
        opposite.add(alt)
ings = ings.union(opposite)

In [241]:
df['NER'] = df['NER'].apply(lambda x: check_ingredients(x, ings))
print(count)
count = 0

1389696


In [244]:
# df.loc[43992]['NER']
# df.loc[43992]['ingredients']
df.loc[234]['directions']

'["In a large skillet, saute onion and green pepper in butter until tender.", "Add tomatoes, mushrooms, olives and oregano.", "Add ground beef if desired.", "Simmer uncovered for 10 minutes.", "Place half of the spaghetti in a greased 13 x 9 x 2-inch baking dish. Top with half of the vegetable mixture.", "Sprinkle with 1 cup Cheddar cheese.", "Repeat layers."]'

In [73]:
ingredients_df = pd.DataFrame(list(ings), columns=['name'])

In [74]:
recipes_df = df.drop(columns=['ingredients', 'directions', 'source', 'NER']).rename(columns={'title': 'name'})
recipes_df['id'] = df.index
recipes_df = recipes_df[['id', 'name', 'link', 'site']]
recipes_df

,id,name,link,site
0,0,No-Bake Nut Cookies,www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com
1,1,Jewell Ball'S Chicken,www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com
2,2,Creamy Corn,www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com
3,3,Chicken Funny,www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com
4,4,Reeses Cups(Candy),www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com
...,...,...,...,...
2231137,2231137,Sunny's Fake Crepes,www.foodnetwork.com/recipes/sunny-anderson/sun...,www.foodnetwork.com
2231138,2231138,Devil Eggs,cookpad.com/us/recipes/355411-devil-eggs,cookpad.com
2231139,2231139,Extremely Easy and Quick - Namul Daikon Salad,cookpad.com/us/recipes/153324-extremely-easy-a...,cookpad.com
2231140,2231140,Pan-Roasted Pork Chops With Apple Fritters,cooking.nytimes.com/recipes/1015164,cooking.nytimes.com


In [75]:
recipe_ingredients_df = df.explode('NER')
recipe_ingredients_df['recipe_id'] = recipe_ingredients_df.index
recipe_ingredients_df = recipe_ingredients_df.drop(columns=['title', 'ingredients', 'directions', 'link', 'source', 'site']).rename(columns={'NER': 'ingredient'}).reset_index()
recipe_ingredients_df = recipe_ingredients_df[['recipe_id', 'ingredient']]
recipe_ingredients_df

,recipe_id,ingredient
0,0,bite size shredded rice biscuits
1,0,nuts
2,0,brown sugar
3,0,milk
4,0,vanilla
...,...,...
18420498,2231141,egg whites
18420499,2231141,flat-leaf
18420500,2231141,tomato paste
18420501,2231141,ground veal


In [127]:
df[df['NER'].apply(lambda x: 'rice' in x)]
# 'rice' in df.loc[2393]['NER']

,title,ingredients,directions,link,source,NER,site
63,Chicken Casserole,"[""1 can cream of mushroom soup"", ""1 can cream ...","[""Mix all ingredients together in baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=665397,Gathered,"{rice, cream of chicken soup, chicken, cream o...",www.cookbooks.com
76,Consomme Chicken,"[""chicken parts"", ""1 can cream of chicken soup...","[""Add enough liquid to chicken soup and consom...",www.cookbooks.com/Recipe-Details.aspx?id=1078857,Gathered,"{consomme, rice, pepper, cream of chicken soup...",www.cookbooks.com
83,Brown Rice,"[""1 c. raw rice"", ""1 can beef consomme"", ""1 ca...","[""Put in a covered 1 1/2-quart dish."", ""Bake 1...",www.cookbooks.com/Recipe-Details.aspx?id=330188,Gathered,"{mushrooms, rice, margarine, onion soup, beef ...",www.cookbooks.com
131,Chicken Casserole,"[""1/2 c. raw rice"", ""1 can French onion soup"",...","[""Mix all together; pour into a 7 x 12-inch ca...",www.cookbooks.com/Recipe-Details.aspx?id=661800,Gathered,"{cream of chicken, mushrooms, rice, bite, onio...",www.cookbooks.com
181,Chinese Hamburger,"[""1 lb. hamburger"", ""2 Tbsp. oil"", ""1 onion, c...","[""Brown hamburger. Add onion and celery until ...",www.cookbooks.com/Recipe-Details.aspx?id=130210,Gathered,"{onion, rice, noodles, pepper, cream of chicke...",www.cookbooks.com
...,...,...,...,...,...,...,...
2230849,Secret Siamese Chicken,"[""1 tablespoon chopped fresh ginger"", ""2 clove...","[""Combine ginger, garlic, soy sauce, sherry, a...",www.cookstr.com/recipes/secret-siamese-chicken,Recipes1M,"{snow peas, rice, garlic, chicken, soy sauce, ...",www.cookstr.com
2230861,Jumbalaya,"[""2 tbsp butter"", ""1 cup bell pepper, chopped""...","[""Melt butter over medium high heat"", ""Add oni...",cookpad.com/us/recipes/333699-jumbalaya,Recipes1M,"{sausage, onion, rice, bell pepper, tabasco sa...",cookpad.com
2230866,Glady's Spanish Rice,"[""1 lb ground beef, cooked and drained"", ""1 sm...","[""brown ground beef and onion together, drain....",www.food.com/recipe/gladys-spanish-rice-360287,Recipes1M,"{onion, rice, ground beef, tomatoes, salt, chi...",www.food.com
2230952,Chicken Fried Rice,"[""1/4 cups Plain Vegetable Oil, Divided Use"", ...","[""Heat a large heavy-bottomed skillet (cast ir...",tastykitchen.com/recipes/main-courses/chicken-...,Recipes1M,"{scallions, ginger, eggs, onion, rice, garlic,...",tastykitchen.com


In [132]:
def clean(ner):
    cleaned_ner = set()
    for x in ner:
        words = x.split()
        filtered_words = list(filter(lambda w: w.isalpha() and w.islower(), words))
        cleaned_ner.add(' '.join(filtered_words).strip())
    return cleaned_ner


In [1]:
cleaned_df = df.iloc[:1000]
cleaned_df['NER'] = cleaned_df['NER'].apply(clean)

NameError: name 'df' is not defined

In [125]:
from spellchecker import SpellChecker

spell = SpellChecker()
misspelled = spell.unknown(['parsey', 'cheesse', 'broc'])

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))
    # Get a list of `likely` options
    print(spell.candidates(word))

cheese
{'cheeses', 'cheese'}
bro
{'brok', 'bros', 'brock', 'bloc', 'croc', 'bro', 'roc', 'brow', 'bronc'}
parley
{'parley', 'parsec', 'parsley', 'parse'}


In [76]:
# ingredients_to_check = {"pepper", "brown rice"}
# mask = df['NER'].apply(lambda x : ingredients_to_check.issubset(set(x)))
# matching_rows = df[mask]

# res = (matching_rows['NER'])
# display(res)